In [1]:
# %load_ext autoreload
# %autoreload 2
# %load_ext line_profiler

In [2]:
# cd ..

In [3]:
%run -i test_cp_gfc.py

[facinv] 2023-11-03 13:15:03,067: -- The FACINV python package --
[facinv] 2023-11-03 13:15:03,072:   LLLGZ, 2023 (c)   
[facinv] 2023-11-03 13:15:03,073: :solution      to ./out
[facinv] 2023-11-03 13:15:03,073: :data          to ./data
[facinv] 2023-11-03 13:15:03,074: :logs and tmps to ./tmp
[facinv] 2023-11-03 13:15:03,490: generating the signature of this problem
[facinv] 2023-11-03 13:15:03,490: {
  "customer_num": 5,
  "data_dir": "/Users/xue/github/facility-loc-inventory/data/data_0401_0inv.xlsx",
  "one_period": false,
  "plant_num": 2,
  "sku_num": 2,
  "warehouse_num": 13
}
[facinv] 2023-11-03 13:15:03,491: current data has not been generated before
[facinv] 2023-11-03 13:15:03,491: creating a temporary cache @./out/data_0401_0inv-5-False-2-2-13.pk
[facinv] 2023-11-03 13:15:40,202: dumping a temporary cache @./out/data_0401_0inv-5-False-2-2-13.pk
setting T0003_T0021 to 300
setting T0015_T0014 to 1100
Cardinal Optimizer v6.5.1. Build date Apr 11 2023
Copyright Cardinal Operat

## relax intermediate nodes

We now relax all intermediate node binary variables,
- this should be better than pure LP relaxation

This gives a `partial LP relaxation`


In [4]:
for k, v in model.variables['select_edge'].items():
    #k(0,edge)
    if k[1].end.type != "C":
        v.setType(COPT.CONTINUOUS)
        print(f"relaxing {k}")

model.solve()

lpval = model.get_model_objval()
os.makedirs(f"sol_lp_{pick_instance}/", exist_ok=True)

relaxing (0, Edge_P0001-GZA1_T0015_(P0001-GZA1, T0015))
relaxing (0, Edge_P0001-GZA2_T0015_(P0001-GZA2, T0015))
relaxing (0, Edge_T0001_T0002_(T0001, T0002))
relaxing (0, Edge_T0001_T0003_(T0001, T0003))
relaxing (0, Edge_T0001_T0014_(T0001, T0014))
relaxing (0, Edge_T0001_T0015_(T0001, T0015))
relaxing (0, Edge_T0001_T0019_(T0001, T0019))
relaxing (0, Edge_T0001_T0020_(T0001, T0020))
relaxing (0, Edge_T0001_T0021_(T0001, T0021))
relaxing (0, Edge_T0001_T0022_(T0001, T0022))
relaxing (0, Edge_T0001_T0023_(T0001, T0023))
relaxing (0, Edge_T0001_T0024_(T0001, T0024))
relaxing (0, Edge_T0001_T0025_(T0001, T0025))
relaxing (0, Edge_T0001_T0026_(T0001, T0026))
relaxing (0, Edge_T0002_T0019_(T0002, T0019))
relaxing (0, Edge_T0002_T0020_(T0002, T0020))
relaxing (0, Edge_T0002_T0021_(T0002, T0021))
relaxing (0, Edge_T0002_T0022_(T0002, T0022))
relaxing (0, Edge_T0002_T0023_(T0002, T0023))
relaxing (0, Edge_T0003_T0001_(T0003, T0001))
relaxing (0, Edge_T0003_T0002_(T0003, T0002))
relaxing (0, E

In [ ]:

from tqdm.notebook import tqdm as tqdm_notebook

import utils
from cp_gfc import *
model.model.setParam("Logging", 0)
# get the first solutions
# 重写了下get_solution及存储的表结构
df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
if df_edges.empty:
    print("no solution")
else:
    print('----------Originial Solution----------')
    for _,row in df_edges.iterrows():
        if 0 < row['y'] <1:
            print(row["obj_start"], row["obj_end"], row["t"],row['y'])
    # print(df_edges['y'])
lpval = model.model.objval

# for t in range(arg.T):
#
if not query_fractional_inter_edges(df_edges).empty:
    n = 1
    for _, row in query_fractional_inter_edges(df_edges).iterrows():
        start, end, t = row["obj_start"], row["obj_end"], row["t"]
        print("------------------Fractional Inter Edge------------------")
        print(k, start, end, t)
        k = 0
        while k < 20:
            # break
            for e in utils.get_out_edges(network, end):
                if e.capacity > 1e6:
                    print("无上界约束的边", e)
                    e.capacity = 0.4e5

            N1 = [e for e in utils.get_out_edges(network, end) if e.capacity < 1e6]
            N2 = [e for e in utils.get_in_edges(network, end) if e.capacity < 1e6]
            x = model.variables["sku_flow"]
            y = model.variables["select_edge"]
            d = 0
            print('---------Seperation Start----------')
            cutv, lbdv, subsets = seperation_gcf(model, x, y, t, N1, N2, d, dump=False, verbose=False)
            print("lbd", lbdv)
            if cutv > 1e-2:
                # 修改点2
                print('---------Add Cut----------')
                expr, cut_value, lbd, bool_voilate = eval_cut_c3(x, y, t, d, *subsets, lbdv=lbdv)
                model.model.addConstr(expr <= 0)
                model.model.solve()
                df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
            else:
                print("cannot find")
                # print(df_edges['y'])
                for _, row in df_edges.iterrows():
                    if 0 < row['y'] < 1:
                        print(row["obj_start"], row["obj_end"], row["t"], row['y'])
                break
            # 看这一步和上一步相比的提升效果 新增了看这一步和mip之间的效果
            print(f"""
            --- {k} ---
            {start, end, t}
            {model.model.objval - lpval}
            {model.model.objval - mipval}
                """)
            lpval = model.model.objval
            k += 1
        print('After adding cuts for {}th fractional inter edge, then we have:'.format(n))
        for _, row in df_edges.iterrows():
            if 0 < row['y'] < 1:
                print(row["obj_start"], row["obj_end"], row["t"], row['y'])
        n += 1
else:
    print("no fractional inter edges")
    print(df_edges['y'])


--- summary ---------------------------
lp relaxation: 827488.4088411783,
mip          : 827803.618810628,


[facinv] 2023-11-03 13:15:40,673: table warehouse_total_avg_inventory failed
[facinv] 2023-11-03 13:15:40,690: saving finished
----------Originial Solution----------
T0015 T0014 0 0.01107380010720026
T0015 T0014 1 0.0031168398199036586
T0015 T0014 3 0.002579700102906478
T0015 T0014 4 0.022481060013128277
T0015 T0014 6 0.008039711791724727
------------------Fractional Inter Edge------------------
(6, Edge_T0026_T0025_(T0026, T0025)) T0015 T0014 0
---------Seperation Start----------
Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-05
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19 rows, 88 columns and 54 nonzeros
Model fingerprint: 0x

Then we see that

- CG and LP are different
- but they are worse than MIP (as expected)

In [6]:
lpval = model.model.objval
# 注意这里面的cg是没考虑edge的lowerbound的（无论是在pricing里面还是在RMP里面）
# 解更大可能是因为松弛后有的是0.3 正常来讲就是0了 但是现在额外加了新的约束 导致搜索空间变小 所以目标函数变大
print(
    f"""
--- summary ---------------------------
lp relaxation: {lpval},
mip          : {mipval},
cg           : {np_cg.RMP_model.objval}
"""
)


NameError: name 'np_cg' is not defined

## GFC in `partial LP relaxation`
Our goal is to cutoff these solutions

In [ ]:
from tqdm.notebook import tqdm as tqdm_notebook

import utils
from cp_gfc import *

# 反了吧 应该流入是N1 流出是N2吧
- Let `inbound edges` as $N_2$, `out` as $N_1$,
- net inventory as $d$,
we see, 

\begin{align*}
\begin{aligned}
& \sum_{j \in N_1} x_j-\sum_{j \in N_2} x_j \leq d, \\
& l_j y_j \leq x_j \leq u_j y_j, \quad y_j \in\{0,1\}, \quad j \in N_1 \cup N_2
\end{aligned}
\end{align*}

where each $x_j$ is the total flow summarizing all SKUs

In [ ]:
model.model.setParam("Logging", 0)
# get the first solutions
# 重写了下get_solution及存储的表结构
df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
lpval = model.model.objval

In [ ]:
df_edges

In [7]:
from tqdm.notebook import tqdm as tqdm_notebook

import utils
from cp_gfc import *

model.model.setParam("Logging", 0)
# get the first solutions
# 重写了下get_solution及存储的表结构
df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
if df_edges.empty:
    print("no solution")
else:
    print('----------Originial Solution----------')
    for _, row in df_edges.iterrows():
        if 0 < row['y'] < 1:
            print(row["obj_start"], row["obj_end"], row["t"], row['y'])
    # print(df_edges['y'])
lpval = model.model.objval

# for t in range(arg.T):



[facinv] 2023-11-03 13:53:48,035: table warehouse_total_avg_inventory failed
[facinv] 2023-11-03 13:53:48,050: saving finished
----------Originial Solution----------
T0015 T0003 0 0.9999999999999996
T0015 T0003 1 0.9999999999999996
T0015 T0003 3 0.9999999999999996
T0015 T0003 4 0.9999999999999996
T0015 T0003 5 0.9999999999999996
T0015 T0014 0 0.5092955028135911
T0015 T0014 1 0.24128854237327274
T0015 T0014 3 0.19970614903454542
T0015 T0014 4 0.02248106001312964
T0015 T0014 4 0.02248106001312964
T0015 T0014 6 0.6223901295596365
T0015 T0024 4 0.9999999999999977
T0025 C00120540 0 0.9999999999999998
T0025 C00120540 3 0.9999999999999998


In [8]:
if not query_fractional_inter_edges(df_edges).empty:
    n = 1
    for _, row in query_fractional_inter_edges(df_edges).iterrows():
        start, end, t = row["obj_start"], row["obj_end"], row["t"]
        print("------------------Fractional Inter Edge------------------")
        print(k, start, end, t)
        k = 0
        while k < 20:
            # break
            for e in utils.get_out_edges(network, end):
                if e.capacity > 1e6:
                    print("无上界约束的边", e)
                    e.capacity = 0.4e5

            N1 = [e for e in utils.get_out_edges(network, end) if e.capacity < 1e6]
            N2 = [e for e in utils.get_in_edges(network, end) if e.capacity < 1e6]
            x = model.variables["sku_flow"]
            y = model.variables["select_edge"]
            d = 0
            print('---------Seperation Start----------')
            cutv, lbdv, subsets = seperation_gcf(model, x, y, t, N1, N2, d, dump=False, verbose=False)
            print("lbd", lbdv)
            if cutv > 1e-2:
                # 修改点2
                print('---------Add Cut----------')
                expr, cut_value, lbd, bool_voilate = eval_cut_c3(x, y, t, d, *subsets, lbdv=lbdv)
                model.model.addConstr(expr <= 0)
                model.model.solve()
                df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
            else:
                print("cannot find")
                # print(df_edges['y'])
                for _, row in df_edges.iterrows():
                    if 0 < row['y'] < 1:
                        print(row["obj_start"], row["obj_end"], row["t"], row['y'])
                break
            # 看这一步和上一步相比的提升效果 新增了看这一步和mip之间的效果
            print(f"""
            --- {k} ---
            {start, end, t}
            {model.model.objval - lpval}
            {model.model.objval - mipval}
                """)
            lpval = model.model.objval
            k += 1
        print('After adding cuts for {}th fractional inter edge, then we have:'.format(n))
        for _, row in df_edges.iterrows():
            if 0 < row['y'] < 1:
                print(row["obj_start"], row["obj_end"], row["t"], row['y'])
        n += 1
else:
    print("no fractional inter edges")
    print(df_edges['y'])

------------------Fractional Inter Edge------------------
6 T0015 T0003 4
---------Seperation Start----------
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 103 columns and 64 nonzeros
Model fingerprint: 0xa704c1a7
Model has 60 quadratic objective terms
Model has 6 quadratic constraints
Model has 57 general constraints
Variable types: 67 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e-10, 4e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-10, 1e+00]
  Objective range  [1e-10, 4e+04]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e-03, 1e+00]
  RHS range        [1e-10, 1e+00]
  QRHS range       [1e-10, 1e-10]
  GenCon const rng [1e+01, 4e+04]
         Consider reformulating model or setting NumericFocus parameter
     

In [ ]:
# k = 0
# while k < 20:
#     for _, row in query_fractional_inter_edges(df_edges).iterrows():
#         start, end, t = row["obj_start"], row["obj_end"], row["t"]
#         break
#     # # 修改点1
#     for e in utils.get_out_edges(network, end):
#         if e.capacity > 1e6:
#             print(e)
#             # e.capacity = 0.9e6
#             e.capacity = 0.4e5
#
#     N1 = [e for e in utils.get_out_edges(network, end) if e.capacity < 1e6]
#     N2 = [e for e in utils.get_in_edges(network, end) if e.capacity < 1e6]
#     # 这样修改的话会导致seperation_gcf那里需要用capacity乘的地方有问题
#     # N1 = [e for e in utils.get_out_edges(network, end)]
#     # N2 = [e for e in utils.get_in_edges(network, end)]
#     x = model.variables["sku_flow"]
#     y = model.variables["select_edge"]
#     d = 0
#     t = 0
#     cutv, lbdv, subsets = seperation_gcf(model, x, y, t, N1, N2, d, dump=False, verbose=False)
#     print("lbd",lbdv)
#     # 1e-13 能有个0.852713
#     if cutv > 1e-13:
#         # 修改点2
#         expr, cut_value, lbd, bool_voilate = eval_cut_c3(x, y, t, d, *subsets, lbdv=lbdv)
#         model.model.addConstr(expr <= 0)
#         model.model.solve()
#         df_edges = model.get_solution(f"sol_lp_{pick_instance}/")
#         print(df_edges)
#     else:
#         print("cannot find", cutv)
#         break
#     # 看这一步和上一步相比的提升效果 新增了看这一步和mip之间的效果
#     print(f"""
# --- {k} ---
# {start, end, t}
# {model.model.objval - lpval}
# {model.model.objval - mipval}
#     """)
#     lpval = model.model.objval
#     # print(df_edges)
#     k += 1

In [ ]:
subsets

In [ ]:
df_edges

In [ ]:
%tb
